In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
#%%
import os
if not os.path.exists("../small_libs"):
    print("the directory small_libs is created")
    !git clone https://github.com/vincentvigon/small_libs
    %cd small_libs
else:
    print("the directory small_libs is updated")
    !git pull https://github.com/vincentvigon/small_libs

the directory small_libs is updated
From https://github.com/vincentvigon/small_libs
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

from derivator_fft.derivator import Derivator_fft

ModuleNotFoundError: ignored

# Derivator

## Calcul long d'une divergence

In [ ]:
def test_divergence(N,graph_acceleration):
    a0=tf.linspace(0,1,N)
    a1=tf.linspace(0,1,N)
    a2=tf.linspace(0,1,N)
    a3=tf.linspace(0,1,N)

    a0_=a0[:,None,None,None]
    a1_=a1[None,:,None,None]
    a2_=a2[None,None,:,None]
    a3_=a3[None,None,None,:]

    func=lambda a0,a1,a2,a3:a0**2+a1**2+a2**2+a3**2
    div_func=lambda a0,a1,a2,a3: 2*a0+2*a1+2*a2+2*a3

    F=func(a0_,a1_,a2_,a3_)
    div_F=div_func(a0_,a1_,a2_,a3_)

    ti0=time.time()
    derivator_fft=Derivator_fft(F,[0,1,2,3],[1,1,1,1],lambda a0,a1,a2,a3:a0+a1+a2+a3,graph_acceleration=graph_acceleration)
    print("création de l'objet, durée:",time.time()-ti0)

    ti0=time.time()
    div_F_pred=derivator_fft.D(F)
    print("premier appel de la dérivation:",time.time()-ti0)
    ti0=time.time()
    div_F_pred=derivator_fft.D(F)
    print("second appel de la dérivation:",time.time()-ti0)

    error=tf.reduce_mean(tf.abs(div_F-div_F_pred))
    print("error",error)

In [ ]:
%%time
test_divergence(30,True)

0 shape ieme tenseur: [36, 1, 1, 1]
1 shape ieme tenseur: [1, 36, 1, 1]
2 shape ieme tenseur: [1, 1, 36, 1]
3 shape ieme tenseur: [1, 1, 1, 36]
création de l'objet, durée: 0.015306949615478516
traçage de la méthode de dérivation du Derivator_fft, temps de traçage: 0.00017523765563964844
premier appel de la dérivation: 4.697309970855713
traçage de la méthode de dérivation du Derivator_fft, temps de traçage: 0.00018095970153808594
second appel de la dérivation: 4.538576126098633
error tf.Tensor(1.2188206825055057, shape=(), dtype=float64)
CPU times: user 11.1 s, sys: 101 ms, total: 11.2 s
Wall time: 9.27 s


In [ ]:
%%time
test_divergence(30,False)

0 shape ieme tenseur: [36, 1, 1, 1]
1 shape ieme tenseur: [1, 36, 1, 1]
2 shape ieme tenseur: [1, 1, 36, 1]
3 shape ieme tenseur: [1, 1, 1, 36]
création de l'objet, durée: 0.01532602310180664
premier appel de la dérivation: 4.449671983718872
second appel de la dérivation: 4.260905027389526
error tf.Tensor(1.2188206825055057, shape=(), dtype=float64)
CPU times: user 11.2 s, sys: 214 ms, total: 11.4 s
Wall time: 8.75 s
